In [15]:
from collections import defaultdict

In [16]:
f_name = "./song.csv"

lines = []
with open(f_name,'r') as f:
    tlines = f.readlines()
    lines = [i.strip() for i in tlines]

# split into sections
END = "End_track"
chunks = [] # this variable is related to the midi format. Chunks is split based on the characteristics of the 4 channels of the midi file
# I make use of 2 of those channels, namely left and right hands resp. In each sublist will be a list of notes with tempos
temp = []
for l in lines: # this code translates END delineated text blocks into list separated blocks
    temp.append(l)
    if END in l:
        chunks.append(temp)
        temp = []
        
print([len(i) for i in chunks])

PPQ = 96
BPM = 60_000_000 / 500000 # some settings for me to modify tempo of the translated music

def convert_tick_to_ms(tick):
    return int(tick * 60000 / (BPM * PPQ))


[4, 3, 519, 417]


In [17]:
# mapping goes like 1 2 3 4 5 6 7
# then higher octave goes shift 1 = 8
# lower octave goes shift 8 = 1
# 1 is middle C
# then I can play the sounds of 
# from 1 octave below middle C, to 3 keys above 1 octave above middle C
# middle C is defined as note 60 on midi, which means max keys (considering only white keys)
# supported is 48->84

# 	Note Numbers
# C	C#	D	D#	E	F	F#	G	G#	A	A#	B
# -1	0	1	2	3	4	5	6	7	8	9	10	11
# 0	12	13	14	15	16	17	18	19	20	21	22	23
# 1	24	25	26	27	28	29	30	31	32	33	34	35
# 2	36	37	38	39	40	41	42	43	44	45	46	47
# 3	48	49	50	51	52	53	54	55	56	57	58	59
# 4	60	61	62	63	64	65	66	67	68	69	70	71
# 5	72	73	74	75	76	77	78	79	80	81	82	83
# 6	84	85	86	87	88	89	90	91	92	93	94	95
# 7	96	97	98	99	100	101	102	103	104	105	106	107
# 8	108	109	110	111	112	113	114	115	116	117	118	119
# 9	120	121	122	123	124	125	126	127


# note construction
midi_to_piano = dict() # constructs the midi notes into music notes. This is used later to determine octave groups to
# to take advantage of raft piano keys
# data type result is midi key (0->127) -> [octave value (+/- 1 away from middle C), Note (C->B)]
index_to_symbol = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
for i in range(128):
    idx = i%12
    octave = (i // 12) - 1
    midi_to_piano[i] = (octave,index_to_symbol[idx])

symbol_to_white_key_index = [i for i in index_to_symbol if '#' not in i] # mapping from symbol to keyboard 1->7
print(symbol_to_white_key_index)

    

['C', 'D', 'E', 'F', 'G', 'A', 'B']


In [18]:
def columProcess(lst,Verbose=True):
    '''
    processes one signal channel of the midi 'chunks' file
    and returns a dataformat of list<tuple<time, note>>
    '''
    t = dict()
    notetimes = []
    for l in lst:
        l = l.replace(' ','')
        args = l.split(",")
        # print(args)
        if len(args) == 6:
            _,time, event_name, _, note, _ = args
            
            if event_name == "Note_on_c":
                notetimes.append((int(time),int(note)))
    if Verbose:
        notes = [i[1] for i in notetimes]
        print("found",len(notetimes),'notes')
        print("min note: ",min(notes),"max note: ",max(notes))
    return notetimes
            
# chunks[2]

In [19]:
# this chunk of code essentially assembles the data into a format of 
# dict< (time) -> list<note>>

righthand_nt = columProcess(chunks[2])
lefthand_nt = columProcess(chunks[3])
print(lefthand_nt)
timesequence = defaultdict(list)
for t,n in lefthand_nt:
    timesequence[t].append(n)
for t,n in righthand_nt:
    timesequence[t].append(n)

for k in sorted(timesequence.keys()):
    pass
    # print(k,timesequence[k])


found 239 notes
min note:  67 max note:  84
found 188 notes
min note:  53 max note:  57
[(0, 53), (384, 55), (768, 57), (1152, 57), (1536, 53), (1632, 53), (1728, 53), (1824, 53), (1920, 55), (2016, 55), (2112, 55), (2208, 55), (2304, 57), (2496, 57), (2688, 57), (2880, 57), (2976, 57), (3072, 53), (3120, 53), (3168, 53), (3216, 53), (3264, 53), (3312, 53), (3360, 53), (3408, 53), (3456, 55), (3504, 55), (3552, 55), (3600, 55), (3648, 55), (3696, 55), (3744, 55), (3792, 55), (3840, 57), (3888, 57), (3936, 57), (3984, 57), (4032, 57), (4080, 57), (4128, 57), (4176, 57), (4224, 57), (4608, 53), (4704, 53), (4752, 53), (4800, 53), (4896, 53), (4944, 53), (4992, 55), (5088, 55), (5136, 55), (5184, 55), (5280, 55), (5328, 55), (5376, 57), (5472, 57), (5520, 57), (5568, 57), (5664, 57), (5712, 57), (5760, 57), (5856, 57), (5904, 57), (5952, 57), (6048, 57), (6096, 57), (6144, 53), (6240, 53), (6288, 53), (6336, 53), (6432, 53), (6480, 53), (6528, 55), (6624, 55), (6672, 55), (6720, 55), (681

In [20]:


def determine_possible_key_group(piano_key_tuple):
    # takes in sheet music keys and returns possible octaines this key resides in

    # this method is raft specific in that it accounts for the 1-0 layout
    # middle C octave starts at oct = 4
    # For example, middle C coule be represented by (4,1), or (3,8), since 8 mod 7=1
    # this means that whenever raft plays keys involving <shift> or <space>, we want to 
    # try to merge them into without pressing shift or space if possible, so we can press more than
    # 1 key at once
    
    octave, note = piano_key_tuple
    assert('#' not in note)
    possible_octaves = {octave}
    number = symbol_to_white_key_index.index(note) + 1
    if number in {1,2,3}: # can lower a group on raft
        possible_octaves.add(octave-1)
    return possible_octaves
    
def determine_note_number(target_group, piano_key_tuple):
    # takes in sheet music keys and tries to map that to a numeric keyboard number
    octave, note = piano_key_tuple
    number = symbol_to_white_key_index.index(note) + 1
    if target_group == octave-1: # we desire to make use of 8,9,0 keys:
        return 7 + number
    return number


class Press:
    def __init__(self, group, number):
        assert(type(group) is int and type(number) is int)
        self.group = group
        self.number = number
    def __repr__(self):
        return '(' + str(self.group) + ',' + str(self.number) + ')'
    
def verify_octave_range(press):
    assert(press.group in {3,4,5})
    
def compose(sequence):
    # sequence is OrderedDict of time->[key1, key2, ...]
    # key_i is in sheet music note format <octave, note>
    # 4 is normal
    # 5 is shift
    # 3 is space
    # returns orderedDict of time->[Press1, Press2, ...]
    final_result = dict()
    times = sorted(sequence.keys())
    delta_notation = [0]
    prev = 0
    for t in times: # we search all notes of a particular time before moving on to the next time to construct the song
        presses = []
        
        delta_notation.append(t-prev) # this accounts for the gap between notes so we can automate hotkey script writing
        prev = t
        note_octaves = [determine_possible_key_group(midi_to_piano[i]) for i in sequence[t]]
        possible_octaves = set.intersection(*note_octaves)
        # either we can use an intersection, or each has their own
        if len(possible_octaves) > 0:
            # yes there is intersection
            target_group = possible_octaves.pop() # at this moment, the possible octaves is the intersection of viable octaves for all notes
                                                  # meaning that any octave we choose is fine. 
                                                  # This part of the algo can be improved to
                                                  # account for pairwise intersection
            for midi_note in sequence[t]:
                number = determine_note_number(target_group, midi_to_piano[midi_note])
                presses.append(Press(target_group, number))
        else:
            # there are no intersection, each note uses its own octave
            for midi_note in sequence[t]:
                group = determine_possible_key_group(midi_to_piano[midi_note])
                group_num = group.pop()
                number = determine_note_number(group_num, midi_to_piano[midi_note])
                presses.append(Press(group_num, number))
        
        [verify_octave_range(p) for p in presses]
        final_result[t] = presses
    return final_result


presssequence = compose(timesequence)

In [21]:
# sample script
#SingleInstance Force
# SetKeyDelay,30,5

# m::
# d = 1000 ; // this is a comment -> delay in ms
# SendEvent, {RShift down}
# Send, 1
# Sleep, 25
# SendEvent, {RShift up}

# Sleep, d
# Send, 1
# Sleep, d
# Send, {Space} 1 2
# Sleep, d
# Send, 1 
# Sleep, d
# return



def produce_notes_for_duration_t(ticks, presses):
    # t is an int, the allowed duration for pressing these notes
    # presses is a list of Press
    # basic algorithm:
    # if single press at time t, press key and account for time difference
    # if multiple presses at time t, handle presses without delay first, then handle concurrent press with delay
    
    SHIFT_WAIT_TIME_MS = 32
    
    CONCURRENT_KEY_WAIT_TIME_MS = 32
    
    MIDDLE_C_OCT = 4
    result = []
    newline = '\n'
    
    normal_presses = [p for p in presses if p.group == MIDDLE_C_OCT]
    space_presses = [p for p in presses if p.group == MIDDLE_C_OCT - 1]
    shift_presses = [p for p in presses if p.group == MIDDLE_C_OCT + 1]
    timespent_noncriticalwait = 0
    # normal concurrent presses
    for p in normal_presses:
        result.append("Send, "+str(p.number % 10) + newline)
            
    # need to wait couple of ms for spacebar to begin, but don't wait if no normal presses
    if len(normal_presses) > 0 and len(space_presses) > 0:
        result.append("Sleep, "+str(CONCURRENT_KEY_WAIT_TIME_MS) + newline)
        timespent_noncriticalwait += CONCURRENT_KEY_WAIT_TIME_MS
    for p in space_presses:
        result.append("Send, {Space} "+str(p.number% 10) + newline)
    
    # shift presses
    if ((len(normal_presses) > 0 or len(space_presses) > 0) and 
        len(shift_presses) > 0): # if there were notes before
        result.append("Sleep, "+str(CONCURRENT_KEY_WAIT_TIME_MS) + newline)
        timespent_noncriticalwait += CONCURRENT_KEY_WAIT_TIME_MS
    
    if len(shift_presses) > 0:
        result.append("SendEvent, {RShift down}" + newline)
    
        for p in shift_presses:
            result.append("Send, "+str(p.number% 10) + newline)
        result.append("Sleep, "+str(SHIFT_WAIT_TIME_MS) + newline)
        timespent_noncriticalwait += SHIFT_WAIT_TIME_MS
        result.append("SendEvent, {RShift up}" + newline)
    
    result.append("Sleep, "+str(convert_tick_to_ms(ticks) - timespent_noncriticalwait) + 
                  newline)
    return result

def produce_ahk_script(press_sequence):
    final_result = ["SendMode Input  ; Recommended for new scripts due to its superior speed and reliability.\n",
                   "SetWorkingDir %A_ScriptDir%  ; Ensures a consistent starting directory.\n",
                   "#SingleInstance Force\n",
                   "m::\n"]
    times = sorted(press_sequence.keys())
    delta_notation = []
    presses_list_list = []
    prev = None
    for t in times: # this loop just computes the delta between simultaneous presses to assemble ideal waiting times
                    # so we may process based on ideal waiting time later when we produce key presses
        presses_list_list.append(press_sequence[t])
        if prev is not None:
            delta_notation.append(t-prev)
        prev = t
    delta_notation.append(100) # ticks
    
    for delta_ticks, presses in zip(delta_notation,presses_list_list):
        final_result.extend(produce_notes_for_duration_t(delta_ticks,presses)) # will assemble notes into presses
    
    # print(delta_notation)
    final_result.append("return")
    return final_result
# presssequence
script = produce_ahk_script(presssequence)

In [22]:
with open("./song_script.ahk",'w') as f:
    f.writelines(script)